In [1]:
import tkinter as tk
from tkinter import filedialog, Text
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

class TextPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Predictor")
        self.file_path = None
        
        self.upload_btn = tk.Button(root, text="Upload Dataset", command=self.upload_file)
        self.upload_btn.pack()

        self.train_btn = tk.Button(root, text="Train Model", command=self.train_model, state=tk.DISABLED)
        self.train_btn.pack()

        self.entry_label = tk.Label(root, text="Enter your text:")
        self.entry_label.pack()
        
        self.text_entry = tk.Entry(root, width=50)
        self.text_entry.pack()

        self.predict_btn = tk.Button(root, text="Predict Next Word", command=self.predict_next_word, state=tk.DISABLED)
        self.predict_btn.pack()

        self.result_label = tk.Label(root, text="")
        self.result_label.pack()

    def upload_file(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
        if self.file_path:
            self.train_btn.config(state=tk.NORMAL)

    def train_model(self):
        if self.file_path:
            with open(self.file_path, "r", encoding="utf8") as file:
                lines = file.readlines()

            data = ' '.join(lines)
            data = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', ' ').replace('”', ' ')
            data = ' '.join(data.split())

            tokenizer = Tokenizer()
            tokenizer.fit_on_texts([data])
            self.tokenizer = tokenizer

            with open('token.pkl', 'wb') as token_file:
                pickle.dump(tokenizer, token_file)

            sequence_data = tokenizer.texts_to_sequences([data])[0]
            vocab_size = len(tokenizer.word_index) + 1

            sequences = []
            for i in range(3, len(sequence_data)):
                words = sequence_data[i-3:i+1]
                sequences.append(words)

            sequences = np.array(sequences)
            X = sequences[:, :-1]
            y = to_categorical(sequences[:, -1], num_classes=vocab_size)

            self.model = Sequential()
            self.model.add(Embedding(vocab_size, 10, input_length=3))
            self.model.add(LSTM(1000, return_sequences=True))
            self.model.add(LSTM(1000))
            self.model.add(Dense(1000, activation="relu"))
            self.model.add(Dense(vocab_size, activation="softmax"))
            self.model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

            self.model.fit(X, y, epochs=5, batch_size=64)
            self.model.save('next_words.h5')

            self.predict_btn.config(state=tk.NORMAL)
            self.result_label.config(text="Model trained successfully!")

    def predict_next_word(self):
        text = self.text_entry.get().strip()
        if text:
            words = text.split()[-3:]
            sequence = self.tokenizer.texts_to_sequences([words])[0]
            sequence = np.array([sequence])

            preds = np.argmax(self.model.predict(sequence))
            predicted_word = ""

            for key, value in self.tokenizer.word_index.items():
                if value == preds:
                    predicted_word = key
                    break

            self.result_label.config(text=f"Predicted next word: {predicted_word}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TextPredictorApp(root)
    root.mainloop()


E:\Games\Anaconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
482/482 ━━━━━━━━━━━━━━━━━━━━ 149s 293ms/step - loss: 6.6840
Epoch 2/5
482/482 ━━━━━━━━━━━━━━━━━━━━ 140s 291ms/step - loss: 5.9417
Epoch 3/5
482/482 ━━━━━━━━━━━━━━━━━━━━ 146s 303ms/step - loss: 5.5324
Epoch 4/5
482/482 ━━━━━━━━━━━━━━━━━━━━ 142s 294ms/step - loss: 5.1617
Epoch 5/5
482/482 ━━━━━━━━━━━━━━━━━━━━ 136s 282ms/step - loss: 4.8980


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [1]:
import tkinter as tk
from tkinter import filedialog
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

class ModelTrainerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Model Trainer")
        self.file_path = None
        
        self.upload_btn = tk.Button(root, text="Upload Dataset", command=self.upload_file)
        self.upload_btn.pack()

        self.train_btn = tk.Button(root, text="Train Model", command=self.train_model, state=tk.DISABLED)
        self.train_btn.pack()

        self.result_label = tk.Label(root, text="")
        self.result_label.pack()

    def upload_file(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("Text Files", "*.txt")])
        if self.file_path:
            self.train_btn.config(state=tk.NORMAL)

    def train_model(self):
        if self.file_path:
            with open(self.file_path, "r", encoding="utf8") as file:
                lines = file.readlines()

            data = ' '.join(lines)
            data = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', ' ').replace('”', ' ')
            data = ' '.join(data.split())

            tokenizer = Tokenizer()
            tokenizer.fit_on_texts([data])
            self.tokenizer = tokenizer

            with open('token.pkl', 'wb') as token_file:
                pickle.dump(tokenizer, token_file)

            sequence_data = tokenizer.texts_to_sequences([data])[0]
            vocab_size = len(tokenizer.word_index) + 1

            sequences = []
            for i in range(3, len(sequence_data)):
                words = sequence_data[i-3:i+1]
                sequences.append(words)

            sequences = np.array(sequences)
            X = sequences[:, :-1]
            y = to_categorical(sequences[:, -1], num_classes=vocab_size)

            self.model = Sequential()
            self.model.add(Embedding(vocab_size, 10, input_length=3))
            self.model.add(LSTM(1000, return_sequences=True))
            self.model.add(LSTM(1000))
            self.model.add(Dense(1000, activation="relu"))
            self.model.add(Dense(vocab_size, activation="softmax"))
            self.model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

            self.model.fit(X, y, epochs=70, batch_size=64)
            self.model.save('next_words.h5')

            self.result_label.config(text="Model trained and saved successfully!")

if __name__ == "__main__":
    root = tk.Tk()
    app = ModelTrainerApp(root)
    root.mainloop()


In [2]:
import tkinter as tk
import numpy as np
import pickle
from tensorflow.keras.models import load_model

class TextPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Predictor")

        self.entry_label = tk.Label(root, text="Enter your text:")
        self.entry_label.pack()
        
        self.text_entry = tk.Entry(root, width=50)
        self.text_entry.pack()

        self.predict_btn = tk.Button(root, text="Predict Next Word", command=self.predict_next_word)
        self.predict_btn.pack()

        self.result_label = tk.Label(root, text="")
        self.result_label.pack()
        
        
        self.model = load_model('next_words.h5')
        with open('token.pkl', 'rb') as token_file:
            self.tokenizer = pickle.load(token_file)

    def predict_next_word(self):
        text = self.text_entry.get().strip()
        if text:
            words = text.split()[-3:]
            sequence = self.tokenizer.texts_to_sequences([words])[0]
            sequence = np.array([sequence])

            preds = np.argmax(self.model.predict(sequence))
            predicted_word = ""

            for key, value in self.tokenizer.word_index.items():
                if value == preds:
                    predicted_word = key
                    break

            self.result_label.config(text=f"Predicted next word: {predicted_word}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TextPredictorApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [3]:
import tkinter as tk
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from textblob import TextBlob

class TextPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Predictor")

        self.entry_label = tk.Label(root, text="Enter your text:")
        self.entry_label.pack()
        
        self.text_entry = tk.Entry(root, width=50)
        self.text_entry.pack()

        self.predict_btn = tk.Button(root, text="Predict Next Word", command=self.predict_next_word)
        self.predict_btn.pack()

        self.result_label = tk.Label(root, text="")
        self.result_label.pack()
        
        self.model = load_model('next_words.h5')
        with open('token.pkl', 'rb') as token_file:
            self.tokenizer = pickle.load(token_file)

    def predict_next_word(self):
        text = self.text_entry.get().strip()
        if text:
            words = text.split()[-3:]
            sequence = self.tokenizer.texts_to_sequences([words])[0]
            sequence = np.array([sequence])

            preds = np.argmax(self.model.predict(sequence))
            predicted_word = ""

            for key, value in self.tokenizer.word_index.items():
                if value == preds:
                    predicted_word = key
                    break

            
            sentiment = TextBlob(predicted_word).sentiment

            
            sentiment_label = f"Sentiment - Polarity: {sentiment.polarity}, Subjectivity: {sentiment.subjectivity}"
            self.result_label.config(text=f"Predicted next word: {predicted_word}\n{sentiment_label}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TextPredictorApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step


In [1]:
import tkinter as tk
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from textblob import TextBlob
from translate import Translator

class TextPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Predictor")

        self.entry_label = tk.Label(root, text="Enter your text:")
        self.entry_label.pack()
        
        self.text_entry = tk.Entry(root, width=50)
        self.text_entry.pack()

        self.predict_btn = tk.Button(root, text="Predict Next Word", command=self.predict_next_word)
        self.predict_btn.pack()
        
        self.translate_btn = tk.Button(root, text="Translate to Telugu", command=self.translate_word, state=tk.DISABLED)
        self.translate_btn.pack()

        self.result_label = tk.Label(root, text="")
        self.result_label.pack()
        
        self.model = load_model('next_words.h5')
        with open('token.pkl', 'rb') as token_file:
            self.tokenizer = pickle.load(token_file)
        
        self.translator = Translator(to_lang="te")

    def predict_next_word(self):
        text = self.text_entry.get().strip()
        if text:
            words = text.split()[-3:]
            sequence = self.tokenizer.texts_to_sequences([words])[0]
            sequence = np.array([sequence])

            preds = np.argmax(self.model.predict(sequence))
            predicted_word = ""

            for key, value in self.tokenizer.word_index.items():
                if value == preds:
                    predicted_word = key
                    break

            
            sentiment = TextBlob(predicted_word).sentiment

            sentiment_label = f"Sentiment - Polarity: {sentiment.polarity}, Subjectivity: {sentiment.subjectivity}"
            self.result_label.config(text=f"Predicted next word: {predicted_word}\n{sentiment_label}")
            
            
            self.translate_btn.config(state=tk.NORMAL)
            self.predicted_word = predicted_word  

    def translate_word(self):
        if hasattr(self, 'predicted_word'):
            translated_word = self.translator.translate(self.predicted_word)
            self.result_label.config(text=f"Predicted next word: {self.predicted_word}\nTranslation in Telugu: {translated_word}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TextPredictorApp(root)
    root.mainloop()


In [3]:
import tkinter as tk
from tkinter import font as tkfont
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from textblob import TextBlob
from translate import Translator

class TextPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Predictor")
        
        
        self.root.geometry("500x300")
        self.root.configure(bg="#ffffff")  

        
        self.custom_font = tkfont.Font(family="Arial", size=14)

        
        self.create_widgets()

        
        self.model = load_model('next_words.h5')
        with open('token.pkl', 'rb') as token_file:
            self.tokenizer = pickle.load(token_file)
        self.translator = Translator(to_lang="te")

    def create_widgets(self):
        
        self.header_frame = tk.Frame(self.root, bg="#ffffff")
        self.header_frame.pack(pady=20)

        self.google_logo = tk.Label(self.header_frame, text="Google", font=("Arial", 24, "bold"), bg="#ffffff", fg="#4285F4")
        self.google_logo.pack()

        self.entry_label = tk.Label(self.root, text="Enter your text:", font=self.custom_font, bg="#ffffff")
        self.entry_label.pack(pady=(10, 5))

        self.text_entry = tk.Entry(self.root, width=60, font=self.custom_font)
        self.text_entry.pack(pady=(0, 15))

        self.predict_btn = tk.Button(self.root, text="Predict Next Word", command=self.predict_next_word, font=self.custom_font, bg="#4285F4", fg="#ffffff", relief=tk.FLAT)
        self.predict_btn.pack(pady=5)

        self.translate_btn = tk.Button(self.root, text="Translate to Telugu", command=self.translate_word, state=tk.DISABLED, font=self.custom_font, bg="#34A853", fg="#ffffff", relief=tk.FLAT)
        self.translate_btn.pack(pady=5)

        self.result_label = tk.Label(self.root, text="", font=self.custom_font, bg="#ffffff")
        self.result_label.pack(pady=(20, 0))

    def predict_next_word(self):
        text = self.text_entry.get().strip()
        if text:
            words = text.split()[-3:]
            sequence = self.tokenizer.texts_to_sequences([words])[0]
            sequence = np.array([sequence])

            preds = np.argmax(self.model.predict(sequence))
            predicted_word = ""

            for key, value in self.tokenizer.word_index.items():
                if value == preds:
                    predicted_word = key
                    break

            sentiment = TextBlob(predicted_word).sentiment

            sentiment_label = f"Sentiment - Polarity: {sentiment.polarity:.2f}, Subjectivity: {sentiment.subjectivity:.2f}"
            self.result_label.config(text=f"Predicted next word: {predicted_word}\n{sentiment_label}")

            self.translate_btn.config(state=tk.NORMAL)
            self.predicted_word = predicted_word  

    def translate_word(self):
        if hasattr(self, 'predicted_word'):
            translated_word = self.translator.translate(self.predicted_word)
            self.result_label.config(text=f"Predicted next word: {self.predicted_word}\nTranslation in Telugu: {translated_word}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TextPredictorApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step


In [13]:
import tkinter as tk
from tkinter import font as tkfont
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from textblob import TextBlob
from translate import Translator
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class TextPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Predictor")

        self.root.geometry("600x600")
        self.root.configure(bg="#ffffff")

        self.custom_font = tkfont.Font(family="Arial", size=14)

        self.create_widgets()

        self.model = load_model('next_words.h5')
        with open('token.pkl', 'rb') as token_file:
            self.tokenizer = pickle.load(token_file)
        self.translator = Translator(to_lang="te")

    def create_widgets(self):
        self.header_frame = tk.Frame(self.root, bg="#ffffff")
        self.header_frame.pack(pady=20)

        self.google_logo = tk.Label(self.header_frame, text="Word Predictor", font=("Arial", 24, "bold"), bg="#ffffff", fg="#4285F4")
        self.google_logo.pack()

        self.entry_label = tk.Label(self.root, text="Enter your text:", font=self.custom_font, bg="#ffffff")
        self.entry_label.pack(pady=(10, 5))

        self.text_entry = tk.Entry(self.root, width=60, font=self.custom_font)
        self.text_entry.pack(pady=(0, 15))

        self.predict_btn = tk.Button(self.root, text="Predict Next Word", command=self.predict_next_word, font=self.custom_font, bg="#4285F4", fg="#ffffff", relief=tk.FLAT)
        self.predict_btn.pack(pady=5)

        self.translate_btn = tk.Button(self.root, text="Translate to Telugu", command=self.translate_word, state=tk.DISABLED, font=self.custom_font, bg="#34A853", fg="#ffffff", relief=tk.FLAT)
        self.translate_btn.pack(pady=5)

        self.result_label = tk.Label(self.root, text="", font=self.custom_font, bg="#ffffff")
        self.result_label.pack(pady=(20, 0))

        self.related_words_label = tk.Label(self.root, text="", font=self.custom_font, bg="#ffffff")
        self.related_words_label.pack(pady=(20, 0))

    def predict_next_word(self):
        text = self.text_entry.get().strip()
        if text:
            words = text.split()[-3:]
            sequence = self.tokenizer.texts_to_sequences([words])[0]
            sequence = np.array([sequence])

            preds = np.argmax(self.model.predict(sequence))
            predicted_word = ""

            for key, value in self.tokenizer.word_index.items():
                if value == preds:
                    predicted_word = key
                    break

            sentiment = TextBlob(predicted_word).sentiment

            sentiment_label = f"Sentiment - Polarity: {sentiment.polarity:.2f}, Subjectivity: {sentiment.subjectivity:.2f}"
            self.result_label.config(text=f"Predicted next word: {predicted_word}\n{sentiment_label}")

            self.translate_btn.config(state=tk.NORMAL)
            self.predicted_word = predicted_word

            related_words = self.scrape_related_words(predicted_word)
            related_words_tfidf = self.get_related_words_tfidf(predicted_word, related_words)
            related_words_sentiments = self.get_related_words_sentiments(related_words_tfidf)
            self.display_related_words(related_words_sentiments)

    def scrape_related_words(self, query):
        search_url = f"https://www.google.com/search?q={query}"
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, "html.parser")
        search_results = soup.find_all("h3")
        related_words = [result.text for result in search_results]
        return related_words

    def get_related_words_tfidf(self, query, related_words):
        documents = [query] + related_words
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
        cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

        related_words_with_scores = [(word, score) for word, score in zip(related_words, cosine_similarities)]
        related_words_with_scores.sort(key=lambda x: x[1], reverse=True)
        return [word for word, _ in related_words_with_scores]

    def get_related_words_sentiments(self, related_words):
        sentiments = []
        for word in related_words:
            sentiment = TextBlob(word).sentiment
            sentiments.append((word, sentiment.polarity, sentiment.subjectivity))
        return sentiments

    def display_related_words(self, related_words_sentiments):
        text = "Related words and their sentiments:\n"
        for word, polarity, subjectivity in related_words_sentiments:
            text += f"{word} - Polarity: {polarity:.2f}, Subjectivity: {subjectivity:.2f}\n"
        self.related_words_label.config(text=text)

    def translate_word(self):
        if hasattr(self, 'predicted_word'):
            translated_word = self.translator.translate(self.predicted_word)
            self.result_label.config(text=f"Predicted next word: {self.predicted_word}\nTranslation in Telugu: {translated_word}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TextPredictorApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 406ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


In [14]:
pip install SpeechRecognition pyaudio


   ---------------------------------------- 0.0/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/32.8 MB 9.6 MB/s eta 0:00:04
   ---------------------------------------- 0.4/32.8 MB 5.7 MB/s eta 0:00:06
    --------------------------------------- 0.7/32.8 MB 5.9 MB/s eta 0:00:06
   - -------------------------------------- 1.1/32.8 MB 6.9 MB/s eta 0:00:05
   - -------------------------------------- 1.4/32.8 MB 6.9 MB/s eta 0:00:05
   -- ------------------------------------- 1.7/32.8 MB 6.9 MB/s eta 0:00:05
   -- ------------------------------------- 2.0/32.8 MB 6.9 MB/s eta 0:00:05
   --- ------------------------------------ 2.5/32.8 MB 7.3 MB/s eta 0:00:05
   --- ------------------------------------ 2.7/32.8 MB 6.9 MB/s eta 0:00:05
   --- ------------------------------------ 2.8/32.8 MB 6.5 MB/s eta 0:00:05
   --- ------------------------------------ 3.2/32.8 MB 6.5 MB/s eta 0:00:05
   ---- ----------------------------------- 3.3/32.8 MB 6.2 MB/s eta 0:00:05
   --

In [15]:
import tkinter as tk
from tkinter import font as tkfont
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from textblob import TextBlob
from translate import Translator
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import speech_recognition as sr

class TextPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Predictor")

        self.root.geometry("600x600")
        self.root.configure(bg="#ffffff")

        self.custom_font = tkfont.Font(family="Arial", size=14)

        self.create_widgets()

        self.model = load_model('next_words.h5')
        with open('token.pkl', 'rb') as token_file:
            self.tokenizer = pickle.load(token_file)
        self.translator = Translator(to_lang="te")

    def create_widgets(self):
        self.header_frame = tk.Frame(self.root, bg="#ffffff")
        self.header_frame.pack(pady=20)

        self.google_logo = tk.Label(self.header_frame, text="Word Predictor", font=("Arial", 24, "bold"), bg="#ffffff", fg="#4285F4")
        self.google_logo.pack()

        self.entry_label = tk.Label(self.root, text="Enter your text or use voice input:", font=self.custom_font, bg="#ffffff")
        self.entry_label.pack(pady=(10, 5))

        self.text_entry = tk.Entry(self.root, width=60, font=self.custom_font)
        self.text_entry.pack(pady=(0, 15))

        self.voice_btn = tk.Button(self.root, text="Voice Input", command=self.voice_input, font=self.custom_font, bg="#FBBC05", fg="#ffffff", relief=tk.FLAT)
        self.voice_btn.pack(pady=5)

        self.predict_btn = tk.Button(self.root, text="Predict Next Word", command=self.predict_next_word, font=self.custom_font, bg="#4285F4", fg="#ffffff", relief=tk.FLAT)
        self.predict_btn.pack(pady=5)

        self.translate_btn = tk.Button(self.root, text="Translate to Telugu", command=self.translate_word, state=tk.DISABLED, font=self.custom_font, bg="#34A853", fg="#ffffff", relief=tk.FLAT)
        self.translate_btn.pack(pady=5)

        self.result_label = tk.Label(self.root, text="", font=self.custom_font, bg="#ffffff")
        self.result_label.pack(pady=(20, 0))

        self.related_words_label = tk.Label(self.root, text="", font=self.custom_font, bg="#ffffff")
        self.related_words_label.pack(pady=(20, 0))

    def voice_input(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            self.result_label.config(text="Listening...")
            audio = recognizer.listen(source)
            try:
                text = recognizer.recognize_google(audio)
                self.text_entry.delete(0, tk.END)
                self.text_entry.insert(0, text)
                self.result_label.config(text="Voice input received. You can now predict the next word.")
            except sr.UnknownValueError:
                self.result_label.config(text="Sorry, could not understand the audio.")
            except sr.RequestError:
                self.result_label.config(text="Could not request results; check your network connection.")

    def predict_next_word(self):
        text = self.text_entry.get().strip()
        if text:
            words = text.split()[-3:]
            sequence = self.tokenizer.texts_to_sequences([words])[0]
            sequence = np.array([sequence])

            preds = np.argmax(self.model.predict(sequence))
            predicted_word = ""

            for key, value in self.tokenizer.word_index.items():
                if value == preds:
                    predicted_word = key
                    break

            sentiment = TextBlob(predicted_word).sentiment

            sentiment_label = f"Sentiment - Polarity: {sentiment.polarity:.2f}, Subjectivity: {sentiment.subjectivity:.2f}"
            self.result_label.config(text=f"Predicted next word: {predicted_word}\n{sentiment_label}")

            self.translate_btn.config(state=tk.NORMAL)
            self.predicted_word = predicted_word

            related_words = self.scrape_related_words(predicted_word)
            related_words_tfidf = self.get_related_words_tfidf(predicted_word, related_words)
            related_words_sentiments = self.get_related_words_sentiments(related_words_tfidf)
            self.display_related_words(related_words_sentiments)

    def scrape_related_words(self, query):
        search_url = f"https://www.google.com/search?q={query}"
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, "html.parser")
        search_results = soup.find_all("h3")
        related_words = [result.text for result in search_results]
        return related_words

    def get_related_words_tfidf(self, query, related_words):
        documents = [query] + related_words
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
        cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

        related_words_with_scores = [(word, score) for word, score in zip(related_words, cosine_similarities)]
        related_words_with_scores.sort(key=lambda x: x[1], reverse=True)
        return [word for word, _ in related_words_with_scores]

    def get_related_words_sentiments(self, related_words):
        sentiments = []
        for word in related_words:
            sentiment = TextBlob(word).sentiment
            sentiments.append((word, sentiment.polarity, sentiment.subjectivity))
        return sentiments

    def display_related_words(self, related_words_sentiments):
        text = "Related words and their sentiments:\n"
        for word, polarity, subjectivity in related_words_sentiments:
            text += f"{word} - Polarity: {polarity:.2f}, Subjectivity: {subjectivity:.2f}\n"
        self.related_words_label.config(text=text)

    def translate_word(self):
        if hasattr(self, 'predicted_word'):
            translated_word = self.translator.translate(self.predicted_word)
            self.result_label.config(text=f"Predicted next word: {self.predicted_word}\nTranslation in Telugu: {translated_word}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TextPredictorApp(root)
    root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [16]:
pip install pyttsx3

   ---------------------------------------- 0.0/219.4 kB ? eta -:--:--
   ---------------------------------------  215.0/219.4 kB 6.6 MB/s eta 0:00:01
   ---------------------------------------- 219.4/219.4 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tkinter as tk
from tkinter import font as tkfont
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from textblob import TextBlob
from translate import Translator
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import speech_recognition as sr
import pyttsx3

class TextPredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text Predictor")

        self.root.geometry("600x600")
        self.root.configure(bg="#ffffff")

        self.custom_font = tkfont.Font(family="Arial", size=14)

        self.create_widgets()

        self.model = load_model('next_words.h5')
        with open('token.pkl', 'rb') as token_file:
            self.tokenizer = pickle.load(token_file)
        self.translator = Translator(to_lang="te")

        self.engine = pyttsx3.init()

    def create_widgets(self):
        self.header_frame = tk.Frame(self.root, bg="#ffffff")
        self.header_frame.pack(pady=20)

        self.google_logo = tk.Label(self.header_frame, text="Word Predictor", font=("Arial", 24, "bold"), bg="#ffffff", fg="#4285F4")
        self.google_logo.pack()

        self.entry_label = tk.Label(self.root, text="Enter your text or use voice input:", font=self.custom_font, bg="#ffffff")
        self.entry_label.pack(pady=(10, 5))

        self.text_entry = tk.Entry(self.root, width=60, font=self.custom_font)
        self.text_entry.pack(pady=(0, 15))

        self.voice_btn = tk.Button(self.root, text="Voice Input", command=self.voice_input, font=self.custom_font, bg="#FBBC05", fg="#ffffff", relief=tk.FLAT)
        self.voice_btn.pack(pady=5)

        self.predict_btn = tk.Button(self.root, text="Predict Next Word", command=self.predict_next_word, font=self.custom_font, bg="#4285F4", fg="#ffffff", relief=tk.FLAT)
        self.predict_btn.pack(pady=5)

        self.translate_btn = tk.Button(self.root, text="Translate to Telugu", command=self.translate_word, state=tk.DISABLED, font=self.custom_font, bg="#34A853", fg="#ffffff", relief=tk.FLAT)
        self.translate_btn.pack(pady=5)

        self.read_output_btn = tk.Button(self.root, text="Read Output", command=self.read_output, state=tk.DISABLED, font=self.custom_font, bg="#EA4335", fg="#ffffff", relief=tk.FLAT)
        self.read_output_btn.pack(pady=5)

        self.result_label = tk.Label(self.root, text="", font=self.custom_font, bg="#ffffff")
        self.result_label.pack(pady=(20, 0))

        self.related_words_label = tk.Label(self.root, text="", font=self.custom_font, bg="#ffffff")
        self.related_words_label.pack(pady=(20, 0))

    def voice_input(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            self.result_label.config(text="Listening...")
            audio = recognizer.listen(source)
            try:
                text = recognizer.recognize_google(audio)
                self.text_entry.delete(0, tk.END)
                self.text_entry.insert(0, text)
                self.result_label.config(text="Voice input received. You can now predict the next word.")
            except sr.UnknownValueError:
                self.result_label.config(text="Sorry, could not understand the audio.")
            except sr.RequestError:
                self.result_label.config(text="Could not request results; check your network connection.")

    def predict_next_word(self):
        text = self.text_entry.get().strip()
        if text:
            words = text.split()[-3:]
            sequence = self.tokenizer.texts_to_sequences([words])[0]
            sequence = np.array([sequence])

            preds = np.argmax(self.model.predict(sequence))
            predicted_word = ""

            for key, value in self.tokenizer.word_index.items():
                if value == preds:
                    predicted_word = key
                    break

            sentiment = TextBlob(predicted_word).sentiment

            sentiment_label = f"Sentiment - Polarity: {sentiment.polarity:.2f}, Subjectivity: {sentiment.subjectivity:.2f}"
            self.result_label.config(text=f"Predicted next word: {predicted_word}\n{sentiment_label}")

            self.translate_btn.config(state=tk.NORMAL)
            self.read_output_btn.config(state=tk.NORMAL)
            self.predicted_word = predicted_word

            related_words = self.scrape_related_words(predicted_word)
            related_words_tfidf = self.get_related_words_tfidf(predicted_word, related_words)
            related_words_sentiments = self.get_related_words_sentiments(related_words_tfidf)
            self.display_related_words(related_words_sentiments)

    def scrape_related_words(self, query):
        search_url = f"https://www.google.com/search?q={query}"
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, "html.parser")
        search_results = soup.find_all("h3")
        related_words = [result.text for result in search_results]
        return related_words

    def get_related_words_tfidf(self, query, related_words):
        documents = [query] + related_words
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
        cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

        related_words_with_scores = [(word, score) for word, score in zip(related_words, cosine_similarities)]
        related_words_with_scores.sort(key=lambda x: x[1], reverse=True)
        return [word for word, _ in related_words_with_scores]

    def get_related_words_sentiments(self, related_words):
        sentiments = []
        for word in related_words:
            sentiment = TextBlob(word).sentiment
            sentiments.append((word, sentiment.polarity, sentiment.subjectivity))
        return sentiments

    def display_related_words(self, related_words_sentiments):
        text = "Related words and their sentiments:\n"
        for word, polarity, subjectivity in related_words_sentiments:
            text += f"{word} - Polarity: {polarity:.2f}, Subjectivity: {subjectivity:.2f}\n"
        self.related_words_label.config(text=text)

    def translate_word(self):
        if hasattr(self, 'predicted_word'):
            translated_word = self.translator.translate(self.predicted_word)
            self.result_label.config(text=f"Predicted next word: {self.predicted_word}\nTranslation in Telugu: {translated_word}")

    def read_output(self):
        output_text = self.result_label.cget("text") + "\n" + self.related_words_label.cget("text")
        self.engine.say(output_text)
        self.engine.runAndWait()

if __name__ == "__main__":
    root = tk.Tk()
    app = TextPredictorApp(root)
    root.mainloop()
